# 파이토치(PyTorch)의 nn.Embedding()
임베딩 벡터를 사용하는 방법이 크게 두 가지
- 임베딩 층(embedding layer)을 만들어 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법
- 미리 사전에 훈련된 임베딩 벡터(pre-trained word embedding)들을 가져와 사용하는 방법

## 1. 학습하여 사용

### 임베딩 층은 룩업 테이블
- 임베딩 층의 입력으로 사용하기 위해서 입력 시퀀스의 각 단어들은 모두 정수 인코딩이 되어있어야
- 어떤 단어 → 단어에 부여된 고유한 정수값 → 임베딩 층 통과 → 밀집 벡터

### nn.Embedding()을 사용하지 않고 구현

In [1]:
import torch

In [2]:
train_data = 'you need to know how to code'

In [3]:
word_set = set(train_data.split()) # 중복을 제거한 단어들의 집합인 단어 집합 생성.
vocab = {word: i+2 for i, word in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수 맵핑.
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

{'you': 2, 'know': 3, 'need': 4, 'to': 5, 'how': 6, 'code': 7, '<unk>': 0, '<pad>': 1}


In [4]:
# 단어 집합의 크기만큼의 행을 가지는 테이블 생성.
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],
                               [ 0.0,  0.0,  0.0],
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

In [5]:
# 임의의 샘플 문장
sample = 'you need to run'.split()
idxes=[]
# 각 단어를 정수로 변환
for word in sample:
  try:
    idxes.append(vocab[word])
  except KeyError: # 단어 집합에 없는 단어일 경우 <unk>로 대체된다.
    idxes.append(vocab['<unk>'])
idxes = torch.LongTensor(idxes)

# 룩업 테이블
lookup_result = embedding_table[idxes, :] # 각 정수를 인덱스로 임베딩 테이블에서 값을 가져온다.
print(lookup_result)

tensor([[0.2000, 0.9000, 0.3000],
        [0.2000, 0.1000, 0.8000],
        [0.4000, 0.1000, 0.1000],
        [0.0000, 0.0000, 0.0000]])


### nn.Embedding() 사용

In [6]:
import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings = len(vocab), 
                               embedding_dim = 3,
                               padding_idx = 1)

nn.Embedding은 크게 두 가지 인자를 받음 :  num_embeddings과 embedding_dim입니다.

- num_embeddings : 임베딩을 할 단어들의 개수. 다시 말해 단어 집합의 크기
- embedding_dim : 임베딩 할 벡터의 차원. 사용자가 정해주는 하이퍼파라미터
- padding_idx : 선택적으로 사용하는 인자. 패딩을 위한 토큰의 인덱스


In [7]:
print(embedding_layer.weight)

Parameter containing:
tensor([[-2.8625,  1.7961, -0.1986],
        [ 0.0000,  0.0000,  0.0000],
        [-0.8048,  0.7859,  0.2050],
        [ 0.7183, -1.2849,  0.1338],
        [ 0.3679, -0.4931, -0.5602],
        [-0.5691, -0.3030, -1.0060],
        [ 1.5824, -0.1845, -0.0263],
        [ 0.7893,  1.3310, -1.7532]], requires_grad=True)


## 2.사전 훈련된 워드 임베딩(Pretrained Word Embedding)

### 2.1 IMDB 리뷰 데이터를 훈련 데이터로 사용하기

In [8]:
from torchtext import data, datasets

두 개의 Field 객체를 정의

In [9]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

C:\Users\HunbeomBak\.conda\envs\nlp\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


torchtext.datasets은 IMDB, TREC(질문 분류), 언어 모델링(WikiText-2) 등 다른 여러 데이터셋을 제공   
torchtext.datasets을 사용해 IMDB 데이터셋을 다운로드하고, 이 데이터셋을 학습 데이터셋과 테스트 데이터셋으로 나눔

In [10]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

C:\Users\HunbeomBak\.conda\envs\nlp\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [11]:
print('훈련 데이터의 크기 : {}' .format(len(trainset)))

훈련 데이터의 크기 : 25000


In [12]:
print(vars(trainset[0]))


{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

### 2.2 토치텍스트를 사용한 사전 훈련된 워드 임베딩


#### 2.2.1 사전 훈련된 Word2Vec 모델 확인

In [13]:
from gensim.models import KeyedVectors

 'eng_w2v' 모델을 로드

In [14]:
word2vec_model = KeyedVectors.load_word2vec_format('eng_w2v')


In [15]:
print(word2vec_model['this']) # 영어 단어 'this'의 임베딩 벡터값 출력

[-0.46345907 -1.1891901   1.245648   -0.37404516  1.652432   -0.5652389
 -0.47939372 -0.12096556 -0.11520188 -0.06890516  2.2709115   1.1223398
  1.5259689   1.1311605   0.5108806  -0.23671353  1.0038161  -0.23612505
 -0.28746054 -0.95729315 -0.74801165 -0.7392374  -0.7818662   0.49504504
  0.17349571 -2.360132    0.7226233  -0.15720506 -0.99177873  0.3037044
  1.8142542   0.5037929  -0.9390487   0.98808104 -0.05130086 -0.59606653
 -1.1025496  -0.6204695  -0.5616367   0.46749592 -0.15358241  1.6946322
  1.3283756   0.24887817  0.3837215   1.9791477  -0.93284166  0.75411063
 -0.6865323   0.42933154  1.1695147   2.311801    1.6639466  -0.6908743
  2.5462062  -1.4810184  -0.16076116 -0.47009483  1.0839101  -0.12877102
  0.49614483 -1.7158998   1.303417   -0.24069652  0.4524425  -2.2316554
 -1.0992208  -2.5734618  -0.90478575  0.47986376  0.9350458   0.8525662
  0.74907756  1.6617146  -0.41617632  1.311047    0.06830977 -0.7097945
 -1.2986578   0.01954382 -0.7358456  -0.82133025 -1.7464945

In [16]:
print(word2vec_model['self-indulgent']) # 영어 단어 'self-indulgent'의 임베딩 벡터값 출력


KeyError: "word 'self-indulgent' not in vocabulary"

 Word2Vec 학습시에 존재하지 않았던 단어이므로 'self-indulgent'의 임베딩 벡터값을 갖고 있지 않다는 에러

#### 2.2.2 사전 훈련된 Word2Vec을 초기 임베딩으로 사용
pre-trained된 임베딩 벡터들을 IMDB 리뷰 데이터의 단어들에 맵핑

In [17]:
import torch
import torch.nn as nn
from torchtext.vocab import Vectors

In [18]:
vectors = Vectors(name="eng_w2v") # 사전 훈련된 Word2Vec 모델을 vectors에 저장


100%|█████████████████████████████████████████████████████████████████████████| 21613/21613 [00:00<00:00, 31493.36it/s]


Field 객체의 build_vocab을 통해 훈련 데이터의 단어 집합(vocabulary)를 만드는 것과 동시에 임베딩 벡터값들을 초기화

In [19]:
TEXT.build_vocab(trainset, vectors=vectors, max_size=10000, min_freq=10) # Word2Vec 모델을 임베딩 벡터값으로 초기화


- max_size와 min_freq는 몇 개의 단어들만을 가지고 단어 집합을 생성할 것인지를 정합니다. 
- max_size는 단어 집합의 크기를 제한하고, min_freq=10은 등장 빈도수가 10번 이상인 단어만 허용하는 것한다는 의미입니다. 
- vectors=vectors는 만들어진 단어 집합의 각 단어의 임베딩 벡터값으로 env_w2v에 저장되어져 있던 임베딩 벡터값들로 초기화합니다.

In [ ]:
print(TEXT.vocab.stoi)


In [21]:
print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))


임베딩 벡터의 개수와 차원 : torch.Size([10002, 100]) 


10,002개의 단어 : 훈련 데이터 단어 10000개 + < unk >와 < pad >

In [22]:
print(TEXT.vocab.vectors[0]) # <unk>의 임베딩 벡터값

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


In [23]:
print(TEXT.vocab.vectors[1]) # <pad>의 임베딩 벡터값


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


In [24]:
print(TEXT.vocab.vectors[10]) # this의 임베딩 벡터값

tensor([-0.4635, -1.1892,  1.2456, -0.3740,  1.6524, -0.5652, -0.4794, -0.1210,
        -0.1152, -0.0689,  2.2709,  1.1223,  1.5260,  1.1312,  0.5109, -0.2367,
         1.0038, -0.2361, -0.2875, -0.9573, -0.7480, -0.7392, -0.7819,  0.4950,
         0.1735, -2.3601,  0.7226, -0.1572, -0.9918,  0.3037,  1.8143,  0.5038,
        -0.9390,  0.9881, -0.0513, -0.5961, -1.1025, -0.6205, -0.5616,  0.4675,
        -0.1536,  1.6946,  1.3284,  0.2489,  0.3837,  1.9791, -0.9328,  0.7541,
        -0.6865,  0.4293,  1.1695,  2.3118,  1.6639, -0.6909,  2.5462, -1.4810,
        -0.1608, -0.4701,  1.0839, -0.1288,  0.4961, -1.7159,  1.3034, -0.2407,
         0.4524, -2.2317, -1.0992, -2.5735, -0.9048,  0.4799,  0.9350,  0.8526,
         0.7491,  1.6617, -0.4162,  1.3110,  0.0683, -0.7098, -1.2987,  0.0195,
        -0.7358, -0.8213, -1.7465,  1.5272, -0.1050, -2.0724, -0.1302, -1.4820,
         0.8396,  0.0405, -1.1910, -0.7787,  0.0579, -0.3865,  1.0887,  0.8484,
        -1.4309,  1.7356,  0.2940,  0.62

In [25]:
print(TEXT.vocab.vectors[10000]) # 단어 'self-indulgent'의 임베딩 벡터값

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


이 임베딩 벡터들을 nn.Embedding()의 초기화 입력으로 사용

In [26]:
embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)

In [27]:
print(embedding_layer(torch.LongTensor([10]))) # 단어 this의 임베딩 벡터값


tensor([[-0.4635, -1.1892,  1.2456, -0.3740,  1.6524, -0.5652, -0.4794, -0.1210,
         -0.1152, -0.0689,  2.2709,  1.1223,  1.5260,  1.1312,  0.5109, -0.2367,
          1.0038, -0.2361, -0.2875, -0.9573, -0.7480, -0.7392, -0.7819,  0.4950,
          0.1735, -2.3601,  0.7226, -0.1572, -0.9918,  0.3037,  1.8143,  0.5038,
         -0.9390,  0.9881, -0.0513, -0.5961, -1.1025, -0.6205, -0.5616,  0.4675,
         -0.1536,  1.6946,  1.3284,  0.2489,  0.3837,  1.9791, -0.9328,  0.7541,
         -0.6865,  0.4293,  1.1695,  2.3118,  1.6639, -0.6909,  2.5462, -1.4810,
         -0.1608, -0.4701,  1.0839, -0.1288,  0.4961, -1.7159,  1.3034, -0.2407,
          0.4524, -2.2317, -1.0992, -2.5735, -0.9048,  0.4799,  0.9350,  0.8526,
          0.7491,  1.6617, -0.4162,  1.3110,  0.0683, -0.7098, -1.2987,  0.0195,
         -0.7358, -0.8213, -1.7465,  1.5272, -0.1050, -2.0724, -0.1302, -1.4820,
          0.8396,  0.0405, -1.1910, -0.7787,  0.0579, -0.3865,  1.0887,  0.8484,
         -1.4309,  1.7356,  

### 2.3 토치텍스트에서 제공하는 사전 훈련된 워드 임베딩

- fasttext.en.300d
- fasttext.simple.300d
- glove.42B.300d
- glove.840B.300d
- glove.twitter.27B.25d
- glove.twitter.27B.50d
- glove.twitter.27B.100d
- glove.twitter.27B.200d
- glove.6B.50d
- glove.6B.100d
- glove.6B.200d
- glove.6B.300d <= 이거 사용

In [28]:
from torchtext.vocab import GloVe


Field 객체의 build_vocab을 통해 토치텍스트가 제공하는 사전 훈련된 임베딩 벡터를 다운받을 수 있음

In [29]:
TEXT.build_vocab(trainset, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(trainset)

.vector_cache\glove.6B.zip: 862MB [06:36, 2.17MB/s]                                                                    
100%|██████████████████████████████████████████████████████████████████████▉| 399999/400000 [00:29<00:00, 13528.60it/s]


In [ ]:
print(TEXT.vocab.stoi)


In [30]:
print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))


임베딩 벡터의 개수와 차원 : torch.Size([10002, 300]) 


In [31]:
print(TEXT.vocab.vectors[0]) # <unk>의 임베딩 벡터값


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [32]:
print(TEXT.vocab.vectors[1]) # <pad>의 임베딩 벡터값


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [33]:
print(TEXT.vocab.vectors[10]) # this의 임베딩 벡터값


tensor([-2.0437e-01,  1.6431e-01,  4.1794e-02, -1.3708e-01, -2.9779e-01,
         3.3440e-01, -6.9955e-02, -6.8036e-02,  1.0604e-01, -2.0337e+00,
         1.7977e-01, -7.7403e-02, -1.9518e-01,  1.8324e-01,  3.0017e-02,
        -5.4762e-02, -4.5725e-01, -2.4509e-02,  5.7387e-02, -3.4878e-01,
         3.9696e-02,  4.4826e-01, -5.8462e-02,  4.1181e-01, -3.5411e-02,
        -1.4722e-01,  1.0740e-01, -2.5896e-01, -1.1658e-01,  1.9822e-01,
         3.2850e-01,  2.4177e-01, -5.7177e-01, -5.6442e-02, -9.6437e-01,
         3.4482e-01,  5.4639e-02,  2.3828e-01, -1.9139e-01,  3.0899e-01,
         2.8044e-01, -3.3814e-02, -2.5436e-01,  1.5373e-02,  1.6341e-01,
         2.6352e-01,  1.5812e-01,  3.2044e-01, -2.3082e-01,  2.6050e-01,
         2.0606e-01, -8.9769e-02, -1.0055e-01,  7.0378e-02, -2.7452e-02,
         2.7959e-01, -9.5862e-02,  2.0574e-01,  2.9522e-01, -1.2285e-02,
         1.1164e-01, -5.1373e-02,  4.6106e-01,  2.3014e-02, -3.7141e-01,
        -2.4166e-01,  3.3773e-02,  3.6827e-02,  1.6

In [34]:
print(TEXT.vocab.vectors[9999]) # seeing의 임베딩 벡터값


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

임베딩 벡터들이 저장되어져 있는 TEXT.vocab.vectors를 nn.Embedding()의 초기화 입력으로 사용

In [35]:
embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)


In [36]:
embedding_layer(torch.LongTensor([10])) # 단어 this의 임베딩 벡터값


tensor([[-2.0437e-01,  1.6431e-01,  4.1794e-02, -1.3708e-01, -2.9779e-01,
          3.3440e-01, -6.9955e-02, -6.8036e-02,  1.0604e-01, -2.0337e+00,
          1.7977e-01, -7.7403e-02, -1.9518e-01,  1.8324e-01,  3.0017e-02,
         -5.4762e-02, -4.5725e-01, -2.4509e-02,  5.7387e-02, -3.4878e-01,
          3.9696e-02,  4.4826e-01, -5.8462e-02,  4.1181e-01, -3.5411e-02,
         -1.4722e-01,  1.0740e-01, -2.5896e-01, -1.1658e-01,  1.9822e-01,
          3.2850e-01,  2.4177e-01, -5.7177e-01, -5.6442e-02, -9.6437e-01,
          3.4482e-01,  5.4639e-02,  2.3828e-01, -1.9139e-01,  3.0899e-01,
          2.8044e-01, -3.3814e-02, -2.5436e-01,  1.5373e-02,  1.6341e-01,
          2.6352e-01,  1.5812e-01,  3.2044e-01, -2.3082e-01,  2.6050e-01,
          2.0606e-01, -8.9769e-02, -1.0055e-01,  7.0378e-02, -2.7452e-02,
          2.7959e-01, -9.5862e-02,  2.0574e-01,  2.9522e-01, -1.2285e-02,
          1.1164e-01, -5.1373e-02,  4.6106e-01,  2.3014e-02, -3.7141e-01,
         -2.4166e-01,  3.3773e-02,  3.